In [ ]:
import pandas as pd
import numpy as np

In [ ]:
companies = pd.read_csv('data/company.csv', engine="pyarrow", sep=",")

In [ ]:
compras_bolsa = pd.read_excel('data/Compras_Bolsa_Nacional_(kWh)_2024.xlsx', engine="openpyxl", skiprows=2)
compras_contratos = pd.read_excel('data/Compras_Contrato_(kWh)_2024.xlsx', engine="openpyxl", skiprows=2)


In [ ]:
ccompras_contratos_ = compras_contratos.fillna(0)
ccompras_contratos_ = ccompras_contratos_.drop("Version", axis=1)
ccompras_contratos_ = ccompras_contratos_.set_index(["Fecha", "Código Agente","Mercado"]).stack().reset_index()
ccompras_contratos_.columns = ["date", "agent_code", "mercado","hour", "compras_contrato"]
ccompras_contratos_ = ccompras_contratos_.astype({"hour": int, "compras_contrato":float})
ccompras_contratos_ = ccompras_contratos_.merge(
    companies,
    left_on="agent_code",
    right_on="Values_Code",
    how="left"
)
ccompras_contratos_ = ccompras_contratos_.query("mercado == 'REGULADO'")
ccompras_contratos_["date"] = pd.to_datetime(ccompras_contratos_["date"])

In [ ]:
compras_bolsa_ = compras_bolsa.fillna(0)
compras_bolsa_ = compras_bolsa_.drop("Version", axis=1)
compras_bolsa_ = compras_bolsa_.set_index(["Fecha", "Código Agente",]).stack().reset_index()
compras_bolsa_.columns = ["date", "agent_code", "hour", "compras_bolsa"]
compras_bolsa_ = compras_bolsa_.astype({"hour": int, "compras_bolsa":float})
compras_bolsa_ = compras_bolsa_.merge(
    companies,
    left_on="agent_code",
    right_on="Values_Code",
    how="left"
)

compras_bolsa_["date"] = pd.to_datetime(compras_bolsa_["date"])

In [ ]:
ccompras_contratos_bolsa = ccompras_contratos_.merge(
    compras_bolsa_[["date", "agent_code", "hour", "compras_bolsa"]],
    on=["date", "agent_code", "hour"],
    how="left"
)
ccompras_contratos_bolsa["datetime"] = ccompras_contratos_bolsa["date"] + pd.to_timedelta(ccompras_contratos_bolsa["hour"], unit="h")
ccompras_contratos_bolsa

In [ ]:
ccompras_contratos_bolsa

In [ ]:
ccompras_contratos_bolsa_agg = ccompras_contratos_bolsa.groupby(["agent_code", ccompras_contratos_bolsa["date"].dt.month]).agg({
    # "date": "min",
    "compras_bolsa": "sum",
    "compras_contrato": "sum",
}).reset_index()
ccompras_contratos_bolsa_agg["exposicion_bolsa"] = ccompras_contratos_bolsa_agg["compras_bolsa"].div(ccompras_contratos_bolsa_agg["compras_contrato"] + ccompras_contratos_bolsa_agg["compras_bolsa"])
ccompras_contratos_bolsa_agg

# 2. Precio Bolsa

In [ ]:
pb = pd.read_csv("differences_for_month_4.csv", sep=",", engine="pyarrow", parse_dates=["datetime"])

In [ ]:
ccompras_contratos_bolsa_pb = ccompras_contratos_bolsa.merge(
    pb, on="datetime"
)
ccompras_contratos_bolsa_pb = ccompras_contratos_bolsa_pb.fillna(0)
ccompras_contratos_bolsa_pb

In [ ]:
ccompras_contratos_bolsa_pb["pb_agent_XM"] = ccompras_contratos_bolsa_pb["MPO preideal XM"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model"] = ccompras_contratos_bolsa_pb["MPO preideal Modelo - DERs None"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model_DERS"] = ccompras_contratos_bolsa_pb["MPO preideal Modelo - DERs 2025"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model_bess"] = ccompras_contratos_bolsa_pb["MPO bess_preideal_resource Modelo - DERs None"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb["pb_agent_model_bess_DERS"] = ccompras_contratos_bolsa_pb["MPO bess_preideal_resource Modelo - DERs 2025"] * ccompras_contratos_bolsa_pb["compras_bolsa"]
ccompras_contratos_bolsa_pb

In [ ]:
result = ccompras_contratos_bolsa_pb.groupby("agent_code").agg({
    "pb_agent_XM": "sum",
    "pb_agent_model": "sum",
    "pb_agent_model_DERS": "sum",
    "pb_agent_model_bess": "sum",
    "pb_agent_model_bess_DERS": "sum",
    "compras_bolsa": "sum",
}).reset_index()

result["pb_agent_xm_ratio"] = (result["pb_agent_XM"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_ratio"] = (result["pb_agent_model"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_DERS_ratio"] = (result["pb_agent_model_DERS"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_bess_ratio"] = (result["pb_agent_model_bess"] / result["compras_bolsa"]) * 1E-3
result["pb_agent_model_bess_DERS_ratio"] = (result["pb_agent_model_bess_DERS"] / result["compras_bolsa"]) * 1E-3
result

In [ ]:
result_merge_bolsa_expo = result.merge(
    ccompras_contratos_bolsa_agg.query("date == 4")[["agent_code", "exposicion_bolsa"]],
    on="agent_code", how="left"
)
result_merge_bolsa_expo

In [ ]:
result_merge_bolsa_expo["Gm_xm"] = result_merge_bolsa_expo["pb_agent_xm_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model"] = result_merge_bolsa_expo["pb_agent_model_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model_DERs"] = result_merge_bolsa_expo["pb_agent_model_DERS_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model_BESS"] = result_merge_bolsa_expo["pb_agent_model_bess_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo["Gm_model_BESS_DERs"] = result_merge_bolsa_expo["pb_agent_model_bess_DERS_ratio"] *  result_merge_bolsa_expo["exposicion_bolsa"]  
result_merge_bolsa_expo

In [ ]:
result_merge_bolsa_expo[["agent_code"]+[col for col in result_merge_bolsa_expo if "Gm" in col]].describe()

In [ ]:
result_merge_bolsa_expo[["agent_code"]+[col for col in result_merge_bolsa_expo if "Gm" in col]].head()

In [ ]:
df = pd.read_excel("data/dema_come_agent_SEME1_2024.xlsx", skiprows=2, parse_dates=["Fecha"])


In [ ]:
result_merge_bolsa_expo["delta_bess"] = result_merge_bolsa_expo["Gm_model"] - result_merge_bolsa_expo["Gm_model_BESS"]


In [ ]:
df_ = df.copy()
df_ = df_[
    (df_["Mercado"]=="REGULADO") &
    (df_["Fecha"].dt.month == 4)
]
df_ = df_.set_index(["Fecha", "Codigo Comercializador", "Mercado", "Version"]).sum(axis=1).reset_index()
df_
df_ = df_.reset_index().groupby(["Codigo Comercializador"]).agg({0: "sum"})
result_merge_bolsa_expo = result_merge_bolsa_expo.merge(
    df_.reset_index(),
    left_on="agent_code", right_on="Codigo Comercializador", how="left"
)


In [ ]:
result_merge_bolsa_expo["beneficio"] = abs(result_merge_bolsa_expo["delta_bess"] * result_merge_bolsa_expo[0]).fillna(0)
beneficio = result_merge_bolsa_expo.beneficio.sum()
print(f"Beneficio: {beneficio*1E-6:,.2f} [MCOP/mes]")

In [ ]:
fig = px.scatter(
    result_merge_bolsa_expo,
    x="delta_bess",
    y="exposicion_bolsa",
    color="beneficio",
    # size="beneficio",
)
fig.update_layout({
    "title": "Beneficio vs Exposición Bolsa",
    "xaxis_title": "Beneficio [MCOP/mes]",
    "yaxis_title": "Exposición Bolsa",
})

In [ ]:
result_merge_bolsa_expo.columns.to_list()